In [ ]:
import pandas as pd
df = pd.read_csv('car_fuel_efficiency.csv')
cols = ['engine_displacement','horsepower','vehicle_weight','model_year','fuel_efficiency_mpg']
df_clean = df[cols]
df_clean.isnull().sum()
df_clean['horsepower'].median()

np.float64(149.0)

In [ ]:
import numpy as np
np.random.seed(42)

n = len(df_clean)

n_train = int(0.6 * n)
n_val = int(0.2 * n)
n_test = int(0.2 * n)

idx = np.arange(n)
np.random.shuffle(idx)

df_shuffled = df_clean.iloc[idx]

df_train = df_shuffled.iloc[:n_train].copy()
df_val = df_shuffled.iloc[:n_val].copy()
df_test = df_shuffled.iloc[:n_test].copy()

In [29]:
# Option 1: Fill with 0
mean_value = df_clean['horsepower'].mean()

df_clean_0 = df_clean['horsepower'].fillna(0)
df_clean_mean = df_clean['horsepower'].fillna(mean_value)


In [31]:
import numpy as np

# Feature columns (X) and target (y)
features = ['engine_displacement','horsepower','vehicle_weight','model_year']

# Option 1: Fill missing horsepower with 0
X_train_0 = df_train[features].copy()
X_val_0   = df_val[features].copy()
X_test_0  = df_test[features].copy()

X_train_0['horsepower'] = X_train_0['horsepower'].fillna(0)
X_val_0['horsepower']   = X_val_0['horsepower'].fillna(0)
X_test_0['horsepower']  = X_test_0['horsepower'].fillna(0)

y_train = df_train['fuel_efficiency_mpg'].values
y_val   = df_val['fuel_efficiency_mpg'].values
y_test  = df_test['fuel_efficiency_mpg'].values

# Convert to numpy arrays
X_train_0 = X_train_0.values
X_val_0 = X_val_0.values
X_test_0 = X_test_0.values

# Train model
intercept_0, weights_0 = train_linear_regression(X_train_0, y_train)

# Predict on validation set
X_val_0_aug = np.column_stack([np.ones(X_val_0.shape[0]), X_val_0])
y_pred_val_0 = X_val_0_aug.dot(np.concatenate([[intercept_0], weights_0]))

# Compute RMSE
rmse_val_0 = np.sqrt(np.mean((y_val - y_pred_val_0)**2))
print("Validation RMSE (fill 0):", round(rmse_val_0, 2))


# Option 2: Fill missing horsepower with mean (from training set)
mean_hp = np.mean(X_train_0[:, 1])  # index 1 = horsepower

X_train_mean = X_train_0.copy()
X_val_mean   = X_val_0.copy()
X_test_mean  = X_test_0.copy()

X_train_mean[:, 1] = np.where(np.isnan(X_train_mean[:, 1]), mean_hp, X_train_mean[:, 1])
X_val_mean[:, 1]   = np.where(np.isnan(X_val_mean[:, 1]), mean_hp, X_val_mean[:, 1])
X_test_mean[:, 1]  = np.where(np.isnan(X_test_mean[:, 1]), mean_hp, X_test_mean[:, 1])

# Train model
intercept_mean, weights_mean = train_linear_regression(X_train_mean, y_train)

# Predict on validation set
X_val_mean_aug = np.column_stack([np.ones(X_val_mean.shape[0]), X_val_mean])
y_pred_val_mean = X_val_mean_aug.dot(np.concatenate([[intercept_mean], weights_mean]))

# Compute RMSE
rmse_val_mean = np.sqrt(np.mean((y_val - y_pred_val_mean)**2))
print("Validation RMSE (fill mean):", round(rmse_val_mean, 2))


Validation RMSE (fill 0): 0.52
Validation RMSE (fill mean): 0.52


In [32]:
import numpy as np

# Features and target
features = ['engine_displacement','horsepower','vehicle_weight','model_year']

# Fill missing horsepower with 0
X_train_0 = df_train[features].copy()
X_val_0   = df_val[features].copy()
X_test_0  = df_test[features].copy()

X_train_0['horsepower'] = X_train_0['horsepower'].fillna(0)
X_val_0['horsepower']   = X_val_0['horsepower'].fillna(0)
X_test_0['horsepower']  = X_test_0['horsepower'].fillna(0)

# Convert to numpy arrays
X_train_0 = X_train_0.values
X_val_0   = X_val_0.values
X_test_0  = X_test_0.values

y_train = df_train['fuel_efficiency_mpg'].values
y_val   = df_val['fuel_efficiency_mpg'].values
y_test  = df_test['fuel_efficiency_mpg'].values

# Linear regression function (from your earlier code)
def train_linear_regression(X, y):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])
    XTX = X.T.dot(X)
    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)
    return w[0], w[1:]

# Train model
intercept_0, weights_0 = train_linear_regression(X_train_0, y_train)

# Predict on validation set
X_val_0_aug = np.column_stack([np.ones(X_val_0.shape[0]), X_val_0])
y_pred_val_0 = X_val_0_aug.dot(np.concatenate([[intercept_0], weights_0]))

# Compute RMSE
rmse_val_0 = np.sqrt(np.mean((y_val - y_pred_val_0)**2))
print("Validation RMSE (fill 0):", round(rmse_val_0, 2))


Validation RMSE (fill 0): 0.52


In [33]:
import numpy as np

# Features and target
features = ['engine_displacement','horsepower','vehicle_weight','model_year']

# Fill missing horsepower with 0
X_train = df_train[features].copy().fillna(0).values
X_val   = df_val[features].copy().fillna(0).values

y_train = df_train['fuel_efficiency_mpg'].values
y_val   = df_val['fuel_efficiency_mpg'].values

# Ridge regression function
def train_ridge_regression(X, y, r):
    ones = np.ones(X.shape[0])
    X_aug = np.column_stack([ones, X])
    
    I = np.eye(X_aug.shape[1])
    I[0,0] = 0  # Do not regularize intercept
    
    w = np.linalg.inv(X_aug.T.dot(X_aug) + r * I).dot(X_aug.T).dot(y)
    return w[0], w[1:]

# List of regularization values
r_values = [0, 0.01, 0.1, 1, 5, 10, 100]

# Store RMSEs
rmse_results = {}

for r in r_values:
    intercept, weights = train_ridge_regression(X_train, y_train, r)
    
    # Predict on validation set
    X_val_aug = np.column_stack([np.ones(X_val.shape[0]), X_val])
    y_pred_val = X_val_aug.dot(np.concatenate([[intercept], weights]))
    
    # Compute RMSE
    rmse = np.sqrt(np.mean((y_val - y_pred_val)**2))
    rmse_results[r] = rmse
    print(f"r = {r} | Validation RMSE: {rmse:.3f}")

# Find the best r (minimum RMSE)
best_r = min(rmse_results, key=rmse_results.get)
print(f"\nBest regularization parameter r: {best_r} with RMSE: {rmse_results[best_r]:.3f}")


r = 0 | Validation RMSE: 0.517
r = 0.01 | Validation RMSE: 0.517
r = 0.1 | Validation RMSE: 0.517
r = 1 | Validation RMSE: 0.517
r = 5 | Validation RMSE: 0.517
r = 10 | Validation RMSE: 0.517
r = 100 | Validation RMSE: 0.517

Best regularization parameter r: 100 with RMSE: 0.517


In [ ]:
import numpy as np
import pandas as pd

# Load and clean dataset
df = pd.read_csv('car_fuel_efficiency.csv')
cols = ['engine_displacement','horsepower','vehicle_weight','model_year','fuel_efficiency_mpg']
df_clean = df[cols]

# Linear regression function (no regularization)
def train_linear_regression(X, y):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])
    XTX = X.T.dot(X)
    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)
    return w[0], w[1:]

# Seeds to test
seeds = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
rmse_scores = []

for seed in seeds:
    np.random.seed(seed)
    
    # Shuffle and split dataset
    n = len(df_clean)
    n_train = int(0.6 * n)
    n_val = int(0.2 * n)
    n_test = int(0.2 * n)
    
    idx = np.arange(n)
    np.random.shuffle(idx)
    df_shuffled = df_clean.iloc[idx]
    
    df_train = df_shuffled.iloc[:n_train].copy()
    df_val   = df_shuffled.iloc[n_train:n_train+n_val].copy()
    df_test  = df_shuffled.iloc[n_train+n_val:].copy()
    
    # Features and target
    features = ['engine_displacement','horsepower','vehicle_weight','model_year']
    
    # Fill missing values with 0
    X_train = df_train[features].fillna(0).values
    X_val   = df_val[features].fillna(0).values
    y_train = df_train['fuel_efficiency_mpg'].values
    y_val   = df_val['fuel_efficiency_mpg'].values
    
    # Train linear regression
    intercept, weights = train_linear_regression(X_train, y_train)
    
    # Predict on validation set
    X_val_aug = np.column_stack([np.ones(X_val.shape[0]), X_val])
    y_pred_val = X_val_aug.dot(np.concatenate([[intercept], weights]))
    
    # Compute RMSE
    rmse = np.sqrt(np.mean((y_val - y_pred_val)**2))
    rmse_scores.append(rmse)

# Compute standard deviation of RMSE scores
std_rmse = np.std(rmse_scores)
print("RMSE scores for all seeds:", np.round(rmse_scores, 3))
print("Standard deviation of RMSE:", round(std_rmse, 3))


RMSE scores for all seeds: [0.521 0.522 0.523 0.516 0.511 0.529 0.532 0.51  0.515 0.513]
Standard deviation of RMSE: 0.007


In [36]:
import numpy as np
import pandas as pd

# Load dataset and select relevant columns
df = pd.read_csv('car_fuel_efficiency.csv')
cols = ['engine_displacement','horsepower','vehicle_weight','model_year','fuel_efficiency_mpg']
df_clean = df[cols]

# Use seed 9 to shuffle and split dataset
np.random.seed(9)
n = len(df_clean)
n_train = int(0.6 * n)
n_val = int(0.2 * n)
n_test = int(0.2 * n)

idx = np.arange(n)
np.random.shuffle(idx)
df_shuffled = df_clean.iloc[idx]

df_train = df_shuffled.iloc[:n_train].copy()
df_val   = df_shuffled.iloc[n_train:n_train+n_val].copy()
df_test  = df_shuffled.iloc[n_train+n_val:].copy()

# Combine train and validation sets
df_train_val = pd.concat([df_train, df_val])

# Features and target
features = ['engine_displacement','horsepower','vehicle_weight','model_year']
X_train_val = df_train_val[features].fillna(0).values
y_train_val = df_train_val['fuel_efficiency_mpg'].values

X_test = df_test[features].fillna(0).values
y_test = df_test['fuel_efficiency_mpg'].values

# Ridge regression function
def train_ridge_regression(X, y, r):
    ones = np.ones(X.shape[0])
    X_aug = np.column_stack([ones, X])
    
    I = np.eye(X_aug.shape[1])
    I[0,0] = 0  # Do not regularize intercept
    
    w = np.linalg.inv(X_aug.T.dot(X_aug) + r * I).dot(X_aug.T).dot(y)
    return w[0], w[1:]

# Train model with r = 0.001
r = 0.001
intercept, weights = train_ridge_regression(X_train_val, y_train_val, r)

# Predict on test set
X_test_aug = np.column_stack([np.ones(X_test.shape[0]), X_test])
y_pred_test = X_test_aug.dot(np.concatenate([[intercept], weights]))

# Compute RMSE on test set
rmse_test = np.sqrt(np.mean((y_test - y_pred_test)**2))
print("Test RMSE:", round(rmse_test, 3))


Test RMSE: 0.515
